In [ ]:
import os
import sys

sys.path.insert(0, os.path.abspath("../utils"))
from aoc_utils import load_data, check

In [ ]:
data = load_data(2023, 21)

In [ ]:
# data, part_1, part_2
tests = [
    (
        """...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
...........""",
        16,
        167004,
    ),
    (
        """...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.....S.....
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
...........""",
        None,
        180349,
    ),
    (
        """..........
.....###..
.###.##...
..#.#...#.
....#.#...
.....S....
.##..#....
.......##.
.##.#.###.
..........""",
        None,
        190527,
    ),
    (
        """..........
.....###..
.###.##...
..#.#...#.
....#.#...
.....S....
.##..#....
.......##.
.##.#.###.
.##..##.#.
..........""",
        None,
        184650,
        # A non-square map
        # 6: 27; 10: 72; 50: 1876; 100: 7462; 500: 184650;
    ),
]

# Part 1

In [ ]:
def get_map(data):
    map_ = {}
    for y, line in enumerate(data.splitlines()):
        for x, c in enumerate(line):
            if c == "S":
                start = x, y
                c = "."
            map_[x, y] = c
    return map_, start

In [ ]:
def print_map(map_):
    """Display gardens or step counters maps."""
    min_x = min(x for x, _ in map_)
    max_x = max(x for x, _ in map_)
    min_y = min(y for _, y in map_)
    max_y = max(y for _, y in map_)
    dlen = len(f"{max(map_.values())}")
    for y in range(min_y, max_y + 1):
        print(
            "".join(
                f" {map_[x, y]:{dlen}}" if (x, y) in map_ else " " + "#" * dlen
                for x in range(min_x, max_x + 1)
            )
        )

In [ ]:
def walk(garden, steps, start):
    positions = set([start])
    for _ in range(steps):
        next_positions = set()
        for x, y in positions:
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                next_ = x + dx, y + dy
                if next_ in garden and garden[next_] == ".":
                    next_positions.add(next_)
        positions = next_positions
    return len(positions)

In [ ]:
def reachable(data, steps):
    garden, start = get_map(data)
    return walk(garden, steps, start)

In [ ]:
check(reachable, tests, steps=6)
reachable(data, steps=64)

# Part 2

This solution should work even without a straight line in the starting row and starting column.

This still requires empty borders and square maps (as in the example).

In [ ]:
def walk(garden, steps, start):
    """Compute reachable cells in an infinite garden.

    This is a simple solution that should be reliable.
    To use only with a small number of steps for validation purposes.
    """
    positions = set([start])
    xlen = max(x for x, _ in garden) + 1
    ylen = max(y for _, y in garden) + 1
    for _ in range(steps):
        next_positions = set()
        for x, y in positions:
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                next = x + dx, y + dy
                next_mod = (x + dx) % xlen, (y + dy) % ylen
                if next_mod in garden and garden[next_mod] == ".":
                    next_positions.add(next)
        positions = next_positions
    return len(positions)

In [ ]:
def walk_more(garden, edge_positions):
    """Compute step counters from a walking front."""
    step = min(edge_positions.values())
    map_ = edge_positions.copy()
    next_positions = set()
    added = True
    while added:
        added = False
        positions = next_positions.union(
            set([pos for pos in edge_positions if edge_positions[pos] == step])
        )
        step += 1
        next_positions = set()
        for x, y in positions:
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                next = x + dx, y + dy
                if next in garden and garden[next] == "." and next not in map_:
                    map_[next] = step
                    next_positions.add(next)
                    added = True
    return map_

In [ ]:
def count_reachable(garden, maps, steps, xlen, ylen):
    """Count reachable cells in an infinite garden.

    Requires a square garden with empty borders.
    Empty borders are necessary because the current diagonal fast-forward
    computation supposes that their periodicity starts immediately.
    Only the periodicity of gardens in the starting row and starting column is
    computed so they can have an offset.

    Parameters
    ----------
    garden : Dict[(x, y), "." | "#"]
        The original garden in dictionary form.
    maps : Dict[(i, j), Dict[(x, y), steps]]
        The square of nine maps around the garden (included).
        Values of each maps are shortest path step counters.
    steps : int
        The target number of steps.
    xlen : int
        The width of the central garden.
    ylen : int
        The height of the central garden.

    Returns
    -------
        The number of reachable cells after walking the provided number of steps.
    """
    mod_target = steps % 2
    # central garden
    reached = sum(
        step <= steps and step % 2 == mod_target
        for step in maps[0, 0].values()
    )
    for xdir, ydir in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        map_ = maps[xdir, ydir]
        map_length = abs(xlen * xdir + ylen * ydir)
        aperiodic = True
        while aperiodic:
            reached += sum(
                step <= steps and step % 2 == mod_target
                for step in map_.values()
            )
            # we have periodicity if the borders have a constant xlen - 1 (resp. ylen - 1) difference
            aperiodic = False
            if ydir == 0:
                left = [(0, y) for y in range(map_length)]
                right = [(xlen - 1, y) for y in range(map_length)]
                if xdir == 1:
                    borders = [right, left]
                else:
                    borders = [left, right]
            else:
                assert xdir == 0
                top = [(x, 0) for x in range(map_length)]
                bottom = [(x, ylen - 1) for x in range(map_length)]
                if ydir == 1:
                    borders = [bottom, top]
                else:
                    borders = [top, bottom]
            for p1, p2 in zip(*borders):
                if abs(map_[p1] - map_[p2]) != map_length - 1:
                    aperiodic = True
            if aperiodic:
                map_ = walk_more(
                    garden,
                    {p2: map_[p1] + 1 for p1, p2 in zip(*borders)}
                )
        # fast-forward
        if map_length % 2 == 0:
            step_increase = sum(
                step % 2 == mod_target
                for step in map_.values()
            )
            step_length = map_length
        else:
            # the parity alternates
            step_increase = len(map_)
            step_length = 2 * map_length

        furthest = max(map_.values())
        ff_iterations = (steps - furthest) // step_length
        ds = ff_iterations * step_length
        reached += step_increase * ff_iterations
        while ds < steps:
            ds += map_length
            reached += sum(
                step + ds <= steps and (step + ds) % 2 == mod_target
                for step in map_.values()
            )

    # diagonals for non square maps are a bit more complicated and are not supported
    assert xlen == ylen
    for diag_ in [(-1, -1), (-1, 1), (1, -1), (1, 1)]:
        map_ = maps[diag_]
        if xlen % 2 == 0:
            step_increase = sum(
                step % 2 == mod_target
                for step in map_.values()
            )
            step_length = xlen
        else:
            # we have a 2*2 square of 4 maps
            step_increase = len(map_) * 2
            step_length = 2 * xlen
        furthest = max(map_.values())
        ff_iterations = (steps - furthest) // step_length
        # n is the number of gardens by diagonal
        n = ff_iterations * (step_length // xlen) + 1
        ds = ff_iterations * step_length
        # Sum of n gardens
        reached += step_increase * ff_iterations * (ff_iterations + 1) // 2
        if xlen % 2 == 1:
            # if xlen is odd, we're missing half (rounded up, n is odd) of the last 2*2 diagonal
            # . <- this
            # X X
            # X X . <- this
            # Y Y X X
            # Y Y X X . <- and this
            reached += sum(
                (n + 1) // 2
                for step in map_.values()
                if step + ds <= steps and (step + ds) % 2 == mod_target
            )
            n += 1
            ds += xlen
        while ds < steps:
            reached += sum(
                n
                for step in map_.values()
                if step + ds <= steps and (step + ds) % 2 == mod_target
            )
            n += 1
            ds += xlen
    return reached

In [ ]:
def walk(garden, steps, start):
    """Compute reachable cells in an infinite garden.

    Performs an exhaustive search on a 3x3 grid, and then fast-forward
    if possible to the last iteration.
    """
    positions = set([start])
    xlen = max(x for x, _ in garden) + 1
    ylen = max(y for _, y in garden) + 1
    added = True
    step = 0
    adj_maps = {
        (0, 0): {},
        (-1, 0): {},
        (1, 0): {},
        (0, -1): {},
        (0, 1): {},
        (-1, -1): {},
        (-1, 1): {},
        (1, -1): {},
        (1, 1): {},
    }
    adj_maps[0, 0][start] = 0
    while step < steps and added:
        added = False
        step += 1
        next_positions = set()
        for x, y in positions:
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                next = x + dx, y + dy
                map_x, mod_x = divmod(x + dx, xlen)
                map_y, mod_y = divmod(y + dy, ylen)
                next_mod = mod_x, mod_y
                if (
                    (map_x, map_y) in adj_maps
                    and next_mod in garden
                    and garden[next_mod] == "."
                ):
                    if next_mod not in adj_maps[map_x, map_y]:
                        adj_maps[map_x, map_y][next_mod] = step
                        added = True
                        next_positions.add(next)
        positions = next_positions
    return count_reachable(garden, adj_maps, steps, xlen, ylen)

In [ ]:
def reachable(data, steps):
    garden, start = get_map(data)
    return walk(garden, steps, start)

In [ ]:
check(reachable, tests[:-1], 2, steps=500)
reachable(data, steps=26501365)